In [1]:
import pandas as pd
pd.set_option('max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as stats
sns.set()

%matplotlib inline 


# import sklearn
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
confusion_matrix, f1_score, roc_auc_score, make_scorer)
from sklearn.model_selection import GridSearchCV 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('data/allegations_202007271729.csv')
df.head().T

,0,1,2,3,4
unique_mos_id,10004,10007,10007,10007,10009
first_name,Jonathan,John,John,John,Noemi
last_name,Ruiz,Sears,Sears,Sears,Sierra
command_now,078 PCT,078 PCT,078 PCT,078 PCT,078 PCT
shield_no,8409,5952,5952,5952,24058
complaint_id,42835,24601,24601,26146,40253
month_received,7,11,11,7,8
year_received,2019,2011,2011,2012,2018
month_closed,5,8,8,9,2
year_closed,2020,2012,2012,2013,2019


In [3]:
df['contact_reason'].value_counts()

PD suspected C/V of violation/crime - street                  10078
Other                                                          4104
PD suspected C/V of violation/crime - auto                     2981
PD suspected C/V of violation/crime - bldg                     2542
Moving violation                                               1983
Other violation of VTL                                         1140
Report-dispute                                                 1085
Execution of search warrant                                     913
Report of other crime                                           906
Execution of arrest/bench warrant                               683
Parking violation                                               635
Report-domestic dispute                                         563
C/V intervened on behalf of/observed encounter w/3rd party      540
Report-gun possession/shots fired                               492
PD suspected C/V of violation/crime - subway    

In [4]:
df['allegation'].value_counts()

Physical force                   4849
Word                             3942
Stop                             2300
Search (of person)               2047
Frisk                            1926
                                 ... 
Jewish                              2
Questioned immigration status       1
Other Asian                         1
Sexist Remark                       1
Oriental                            1
Name: allegation, Length: 115, dtype: int64

In [5]:
df.isna().sum()

unique_mos_id                  0
first_name                     0
last_name                      0
command_now                    0
shield_no                      0
complaint_id                   0
month_received                 0
year_received                  0
month_closed                   0
year_closed                    0
command_at_incident         1544
rank_abbrev_incident           0
rank_abbrev_now                0
rank_now                       0
rank_incident                  0
mos_ethnicity                  0
mos_gender                     0
mos_age_incident               0
complainant_ethnicity       4464
complainant_gender          4195
complainant_age_incident    4812
fado_type                      0
allegation                     1
precinct                      24
contact_reason               199
outcome_description           56
board_disposition              0
dtype: int64

In [6]:
df['rank_incident'].value_counts()

Police Officer            22509
Sergeant                   5934
Detective                  3307
Lieutenant                 1301
Captain                     182
Deputy Inspector             96
Inspector                    27
Chiefs and other ranks        2
Name: rank_incident, dtype: int64

In [7]:
df['unique_mos_id'].value_counts()

25861    75
18731    75
19489    73
18530    73
18589    72
         ..
11517     1
26350     1
22637     1
22280     1
5334      1
Name: unique_mos_id, Length: 3996, dtype: int64

In [8]:
df['outcome_description'].value_counts()

No arrest made or summons issued        12822
Arrest - other violation/crime          10196
Summons - disorderly conduct             2118
Summons - other violation/crime          1940
Arrest - resisting arrest                1593
Arrest - disorderly conduct              1013
Arrest - assault (against a PO)           852
Moving violation summons issued           839
Arrest - OGA                              649
Other VTL violation summons issued        531
Parking summons issued                    279
Disorderly-Conduct/Arr/Summons            137
Arrest on Other Charge                     81
Traffic Summons Claimed or Issued          59
Juvenile Report                            57
Other Summons Claimed or Issued            38
Assault/Arrested                           34
Resisting Arrest/Arrested                  25
Arrest - harrassment (against a PO)        15
Obstruct-Govt-Admin/Arrested               10
Harrassment/Arrested/Summons                8
Summons - harrassment (against a P

In [9]:
df['board_disposition'].value_counts()

Unsubstantiated                             15448
Exonerated                                   9609
Substantiated (Charges)                      3796
Substantiated (Formalized Training)          1033
Substantiated (Command Discipline A)          964
Substantiated (Command Discipline)            851
Substantiated (Command Discipline B)          789
Substantiated (Command Lvl Instructions)      454
Substantiated (Instructions)                  248
Substantiated (No Recommendations)            165
Substantiated (MOS Unidentified)                1
Name: board_disposition, dtype: int64

## Drop columns that will not be useful for model 
Not going to use `first_name`, `shiedl_no`, `last_name`, `rank_now`, `rank_incident`. Removing the last two because they're redundant and rank_abbrev have more detail. 

In [10]:
df.drop(['first_name', 'last_name', 'rank_incident', 'rank_now', 'shield_no'], axis=1, inplace=True)

In [11]:
#Sanity check for modified df
df.head().T

,0,1,2,3,4
unique_mos_id,10004,10007,10007,10007,10009
command_now,078 PCT,078 PCT,078 PCT,078 PCT,078 PCT
complaint_id,42835,24601,24601,26146,40253
month_received,7,11,11,7,8
year_received,2019,2011,2011,2012,2018
month_closed,5,8,8,9,2
year_closed,2020,2012,2012,2013,2019
command_at_incident,078 PCT,PBBS,PBBS,PBBS,078 PCT
rank_abbrev_incident,POM,POM,POM,POM,POF
rank_abbrev_now,POM,POM,POM,POM,POF


In [12]:
# check for duplicates 
df.duplicated().sum()

631

In [13]:
# Remove duplicate rows 
print(f'Shape before removing duplicates: {df.shape}')
df.drop_duplicates(inplace=True)
# Sanity check 
print(f'Num duplicates remaining: {df.duplicated().sum()}')
print(f'Shape after removing duplicates: {df.shape}')

Shape before removing duplicates: (33358, 22)
Num duplicates remaining: 0
Shape after removing duplicates: (32727, 22)


In [14]:
# Sort df by date 
df.sort_values(by=['year_received', 'month_received'], inplace=True, ascending=True)

In [15]:
# Sanity check 
df.head().T

,5804,5805,5806,7967,10891
unique_mos_id,17403,17403,17403,18770,20619
command_now,CD OP/D,CD OP/D,CD OP/D,DB CID,FAM SEC
complaint_id,517,517,517,528,525
month_received,9,9,9,10,10
year_received,1985,1985,1985,1985,1985
month_closed,9,9,9,10,10
year_closed,1985,1985,1985,1985,1985
command_at_incident,NaN,NaN,NaN,NaN,NaN
rank_abbrev_incident,POM,POM,POM,POM,POM
rank_abbrev_now,DC,DC,DC,DT2,DTS


In [16]:
df['board_disposition'].value_counts()

Unsubstantiated                             15085
Exonerated                                   9484
Substantiated (Charges)                      3727
Substantiated (Formalized Training)          1014
Substantiated (Command Discipline A)          947
Substantiated (Command Discipline)            843
Substantiated (Command Discipline B)          771
Substantiated (Command Lvl Instructions)      445
Substantiated (Instructions)                  245
Substantiated (No Recommendations)            165
Substantiated (MOS Unidentified)                1
Name: board_disposition, dtype: int64

In [17]:
# create column for unsub/exonerated 
condition = (df['board_disposition']=='Unsubstantiated') | (df['board_disposition']=='Exonerated')
df['unsubst_or_exonerated'] = np.where(condition, 1, 0)

In [18]:
df.head().T

,5804,5805,5806,7967,10891
unique_mos_id,17403,17403,17403,18770,20619
command_now,CD OP/D,CD OP/D,CD OP/D,DB CID,FAM SEC
complaint_id,517,517,517,528,525
month_received,9,9,9,10,10
year_received,1985,1985,1985,1985,1985
month_closed,9,9,9,10,10
year_closed,1985,1985,1985,1985,1985
command_at_incident,NaN,NaN,NaN,NaN,NaN
rank_abbrev_incident,POM,POM,POM,POM,POM
rank_abbrev_now,DC,DC,DC,DT2,DTS


## Split our data into training and testing sets. 
The data will be split based on date. 

In [19]:
# Build a dataframe with the years and how many times they occur in the complaints df
year_val_counts = df['year_received'].value_counts()
df_year_val_counts = pd.DataFrame(year_val_counts)
df_year_val_counts = df_year_val_counts.reset_index()
df_year_val_counts.columns = ['year', 'count']
df_year_val_counts.sort_values(by=['year'], ascending=True, inplace=True)

In [20]:
df_year_val_counts

,year,count
34,1985,7
32,1986,19
33,1987,16
31,1988,32
30,1989,37
29,1990,39
28,1991,41
27,1992,67
26,1993,67
25,1994,153


In [21]:
df_year_val_counts['count'].sum()

32727

## Given data distribution, will create a split the data at 2016 
All complaints starting 2017 will be reserved for test data set. 

In [22]:
# Create a dataframe with training data
df_train = df[df['year_received'] < 2017]
df_test = df[df['year_received'] > 2016] 

In [23]:
# Sanity checks 
df_train.head()

,unique_mos_id,command_now,complaint_id,month_received,year_received,month_closed,year_closed,command_at_incident,rank_abbrev_incident,rank_abbrev_now,mos_ethnicity,mos_gender,mos_age_incident,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition,unsubst_or_exonerated
5804,17403,CD OP/D,517,9,1985,9,1985,NaN,POM,DC,White,M,25,NaN,NaN,NaN,Discourtesy,Curse,83.0,Report of Crime Past/Present,No arrest made or summons issued,Unsubstantiated,1
5805,17403,CD OP/D,517,9,1985,9,1985,NaN,POM,DC,White,M,25,NaN,NaN,NaN,Force,Push/Shove,83.0,Report of Crime Past/Present,No arrest made or summons issued,Unsubstantiated,1
5806,17403,CD OP/D,517,9,1985,9,1985,NaN,POM,DC,White,M,25,NaN,NaN,NaN,Abuse of Authority,Arrest/D. A. T.,83.0,Report of Crime Past/Present,No arrest made or summons issued,Unsubstantiated,1
7967,18770,DB CID,528,10,1985,10,1985,NaN,POM,DT2,White,M,23,NaN,NaN,NaN,Force,Punch/Kick,63.0,Dispute,No arrest made or summons issued,Substantiated (Command Discipline),0
10891,20619,FAM SEC,525,10,1985,10,1985,NaN,POM,DTS,Black,M,23,NaN,NaN,NaN,Force,Dragged/Pulled,71.0,Report of Crime Past/Present,No arrest made or summons issued,Unsubstantiated,1


In [24]:
print(f'Earliest year in training dataframe: {df_train["year_received"].min()} \nLatest year in training dataframe: {df_train["year_received"].max()}')
print(f'Earliest year in testing dataframe: {df_test["year_received"].min()} \nLatest year in testing dataframe: {df_test["year_received"].max()}')

Earliest year in training dataframe: 1985 
Latest year in training dataframe: 2016
Earliest year in testing dataframe: 2017 
Latest year in testing dataframe: 2020


In [25]:
print(f'Training dataframe shape: {df_train.shape} \nTesting dataframe shape: {df_test.shape}')
# make sure dataframe samples add up to the total of the original dataframe
print(df_train.shape[0]+df_test.shape[0]==df.shape[0])

Training dataframe shape: (26827, 23) 
Testing dataframe shape: (5900, 23)
True


## Transform and prep data in each dataframe for modeling 

In [26]:
# Check datatypes 
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26827 entries, 5804 to 33344
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   unique_mos_id             26827 non-null  int64  
 1   command_now               26827 non-null  object 
 2   complaint_id              26827 non-null  int64  
 3   month_received            26827 non-null  int64  
 4   year_received             26827 non-null  int64  
 5   month_closed              26827 non-null  int64  
 6   year_closed               26827 non-null  int64  
 7   command_at_incident       25450 non-null  object 
 8   rank_abbrev_incident      26827 non-null  object 
 9   rank_abbrev_now           26827 non-null  object 
 10  mos_ethnicity             26827 non-null  object 
 11  mos_gender                26827 non-null  object 
 12  mos_age_incident          26827 non-null  int64  
 13  complainant_ethnicity     23337 non-null  object 
 14  com

In [27]:
# check for null values 
df_train.isna().sum()

unique_mos_id                  0
command_now                    0
complaint_id                   0
month_received                 0
year_received                  0
month_closed                   0
year_closed                    0
command_at_incident         1377
rank_abbrev_incident           0
rank_abbrev_now                0
mos_ethnicity                  0
mos_gender                     0
mos_age_incident               0
complainant_ethnicity       3490
complainant_gender          3303
complainant_age_incident    3792
fado_type                      0
allegation                     1
precinct                      13
contact_reason               140
outcome_description            7
board_disposition              0
unsubst_or_exonerated          0
dtype: int64

In [28]:
df_train['complainant_age_incident'].value_counts()

 26.0      903
 24.0      901
 30.0      868
 23.0      851
 25.0      849
          ... 
 87.0        1
-4301.0      1
 7.0         1
 88.0        1
 83.0        1
Name: complainant_age_incident, Length: 88, dtype: int64

In [29]:
def prep_data(df):
    """
    Transforms data to suit modeling needs. Returns df. 
    """
    # Fill null values in numeric rows with median to avoid impact of outliers 
    for label, content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                # Add a column to indicate if data was missing
                df[label+"_is_missing"] = pd.isnull(content)
                # Fill missing numeric values with median
                df[label] = content.fillna(content.median())
    
        # Turn object/string rows into categories 
        if not pd.api.types.is_numeric_dtype(content):
            # Column to indicate if data was missing
            df[label+"_is_missing"] = pd.isnull(content)
            # add +1 to the category code because pandas encodes missing vals with -1
            # This way NaN values will be codes as 1 
            df[label] = pd.Categorical(content).codes+1
    
    return df

In [39]:
# Use prep_data function to categories and fill NaN vals in training df
df_train = prep_data(df_train)

In [41]:
# Use prep_data function to categories and fill NaN vals in testing df
df_test = prep_data(df_test)

## Create and evaluate random forest classification model

In [42]:
df_train.head().T

,5804,5805,5806,7967,10891
unique_mos_id,17403,17403,17403,18770,20619
command_now,169,169,169,192,248
complaint_id,517,517,517,528,525
month_received,9,9,9,10,10
year_received,1985,1985,1985,1985,1985
month_closed,9,9,9,10,10
year_closed,1985,1985,1985,1985,1985
command_at_incident,0,0,0,0,0
rank_abbrev_incident,15,15,15,15,15
rank_abbrev_now,6,6,6,9,11


In [43]:
X_train, y_train = df_train.drop(['outcome_description', 'board_disposition', 'unsubst_or_exonerated'], axis=1), df_train['unsubst_or_exonerated']
X_test, y_test = df_test.drop(['outcome_description', 'board_disposition', 'unsubst_or_exonerated'], axis=1), df_test['unsubst_or_exonerated']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((26827, 35), (26827,), (5900, 35), (5900,))

In [44]:
def evaluate_class_model(model): 
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_true=y_test, y_pred=y_pred, )
    print("Accuracy Score: %f" % accuracy)

    precision = precision_score(y_true=y_test, y_pred=y_pred, average='micro')
    print("Precision Score: %f" % precision)

    recall = recall_score(y_true=y_test, y_pred=y_pred, average='micro')
    print("Recall Score: %f" % recall)

    f1 = f1_score(y_true=y_test, y_pred=y_pred, average='micro')
    print('F1 Score: %f' % f1)

    # Calculate predicted probabilities, keep only probability for when class = 1
#     y_pred_proba = model.predict_proba(X_test)[:,1]
#     auc = roc_auc_score(y_true=y_test, y_score=y_pred_proba, multi_class='ovo')
#     print(f'AUC Score: %f' % auc)

In [45]:
model = RandomForestClassifier(n_estimators=20, criterion='gini', max_features=25, 
                              min_samples_split=2, max_depth=100)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=100, max_features=25, n_estimators=20)

In [46]:
evaluate_class_model(model)

Accuracy Score: 0.615593
Precision Score: 0.615593
Recall Score: 0.615593
F1 Score: 0.615593


In [47]:
params= {'criterion': ['gini', 'entropy'], 
         'max_depth': [2, 4, 5, 10, 20, 100], 
         'max_features': [2, 5, 10, 15, 25, 'auto'], 
         'min_samples_split': [2, 10, 20, 100], 
         'n_estimators': [2, 5, 10, 20, 100],
         }


grid_search_cv =  GridSearchCV( 
    estimator=RandomForestClassifier(), 
    param_grid = params, 
    scoring = 'f1'
    )

In [48]:
grid_search_cv.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 4, 5, 10, 20, 100],
                         'max_features': [2, 5, 10, 15, 25, 'auto'],
                         'min_samples_split': [2, 10, 20, 100],
                         'n_estimators': [2, 5, 10, 20, 100]},
             scoring='f1')

In [49]:
print(grid_search_cv.best_params_)

{'criterion': 'entropy', 'max_depth': 2, 'max_features': 'auto', 'min_samples_split': 100, 'n_estimators': 10}


In [50]:
model = grid_search_cv.best_estimator_
evaluate_class_model(model)

Accuracy Score: 0.718644
Precision Score: 0.718644
Recall Score: 0.718644
F1 Score: 0.718644


In [55]:
# check feature importance 
selected_features = list(df_train.drop(['outcome_description', 'board_disposition', 'unsubst_or_exonerated'], axis=1))
feature_imp = pd.DataFrame.from_dict( {'feature_importance': model.feature_importances_,
                                       'feature':selected_features }).sort_values('feature_importance', ascending=False)
feature_imp

,feature_importance,feature
16,0.276387,fado_type
17,0.140803,allegation
11,0.113405,mos_gender
6,0.111333,year_closed
4,0.071517,year_received
14,0.056398,complainant_gender
7,0.050680,command_at_incident
9,0.044503,rank_abbrev_now
0,0.033355,unique_mos_id
10,0.032833,mos_ethnicity


In [51]:
import pickle
# Dump the trained decision tree classifier with Pickle
filename = 'rf_classifier.pkl'
# Open the file to save as pkl file
pickle.dump(model, open(filename, 'wb'))